In [ ]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from sklearn.cluster import KMeans, DBSCAN

In [ ]:
train_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
# test_df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/test.csv")

In [ ]:
train_df.shape

In [ ]:
train_df.columns

In [ ]:
train_df.index

In [ ]:
train_df.info()

In [ ]:
train_df.head(3)

In [ ]:
#Importing NLTK librariries so that to clean the unstructured language data
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk import word_tokenize
from nltk import pos_tag
import spacy
english_spacy = spacy.load('en_core_web_lg')
from spacy import displacy
import string
import re
nltk.download('stopwords', quiet=True)
stopwords = nltk.corpus.stopwords.words('english')

#removing punctuations and stopwords from unstructured langugae data 
def stopwordsPunctuation_detacher(syntax):
    syntax = re.sub(r'[^\w\s]','',syntax)
    syntax = [lexUnit.lower() for lexUnit in syntax.lower().split() if lexUnit not in stopwords]
    syntax=' '.join(syntax)
    syntax.translate(str.maketrans('','', string.punctuation))
    english_sytax = english_spacy(syntax)
    new_syntax = ' '
    absence=0
    for lex_unit in english_sytax:
        if (lex_unit.is_stop == absence):
            new_syntax = new_syntax + ' ' + str(lex_unit)
    return new_syntax

#removing digits, webLinks from unstructured langugae data and making all words lower case.
def textCleaning_engine(syntax):
    syntax = syntax.lower()
    syntax = re.sub('\[.*?\]', '', syntax)
    syntax = re.sub('https?://\S+|www\.\S+', '', syntax)
    syntax = re.sub('<.*?>+', '', syntax)
    syntax = re.sub('[%s]' % re.escape(string.punctuation), '', syntax)
    syntax = re.sub('\n', '', syntax)
    syntax = re.sub('\w*\d\w*', '', syntax)
    clean_syntax = re.sub(r'\d+','',syntax)
    return clean_syntax
#Combining the two functions above in one function
def ready_sytaxt(syntax):
    syntax = syntax.lower()
    syntax = stopwordsPunctuation_detacher(syntax)
    syntax = textCleaning_engine(syntax)
    return syntax

train_df['dataset_title'] = train_df['dataset_title'].apply(ready_sytaxt)
train_df['pub_title'] = train_df['pub_title'].apply(ready_sytaxt)


In [ ]:
dataset_title_LIST=train_df['dataset_title'].to_list()

National_Education_Longitudinal_Study=[]
Beginning_Postsecondary_Student=[]
Agricultural_Resource_Management_Survey=[]
Baltimore_Longitudinal_StudyOfAging=[]
Coastal_Change_Analysis_Program=[]
Sea_Lake_Overland_Surges_fromHurricanes=[]
NOAA_Tide_Gauge=[]
for i in dataset_title_LIST:
    if i=='National Education Longitudinal Study':
        National_Education_Longitudinal_Study.append(i)
    elif i=='NOAA Tide Gauge':
        NOAA_Tide_Gauge.append(i)
    elif i=='Sea, Lake, and Overland Surges from Hurricanes':
        Sea_Lake_Overland_Surges_fromHurricanes.append(i)
    elif i=='Coastal Change Analysis Program':
        Coastal_Change_Analysis_Program.append(i)
    elif i=="Baltimore Longitudinal Study of Aging (BLSA) ":
        Baltimore_Longitudinal_StudyOfAging.append(i)
    elif i=="Agricultural Resource Management Survey ":
        Agricultural_Resource_Management_Survey.append(i)
    else:
        Beginning_Postsecondary_Student.append(i)
        
# print(National_Education_Longitudinal_Study)
# Beginning_Postsecondary_Student 
# Agricultural_Resource_Management_Survey 
# Baltimore_Longitudinal_StudyOfAging 
# Coastal_Change_Analysis_Program 
# Sea_Lake_Overland_Surges_fromHurricanes
# NOAA_Tide_Gauge


In [ ]:
ready_label=train_df['cleaned_label'].values.astype('str')
train_uniqueLabels = np.unique(ready_label)
labels = [lbl.split() for lbl in train_uniqueLabels]




In [ ]:
def trainANDbuildVocab(label, dimension, epochs):
    word2vec_model = word2vec.Word2Vec(vector_size = dimension, min_count= 1)
    word2vec_model.build_vocab(labels)
    word2vec_model.train(labels, epochs=epochs, total_examples=word2vec_model.corpus_count)
    return word2vec_model


In [ ]:
# glovePretrainedModel = pd.read_csv('../input/glove6b50dtxt/glove.6B.50d.txt', sep=" ", quoting=3, header=None, index_col=0)

In [ ]:
from mxnet import nd
from mxnet.contrib import text
import pickle

def download_gloveAlgorithm():
    print(text.embedding.get_pretrained_file_names('glove'))
    obtain_vector_representations = text.embedding.create('glove', 
                                                          pretrained_file_name='glove.6B.50d.txt')
    return obtain_vector_representations
    
#     with open('glove.6B.50d.pkl', 'wb') as fp:
#         pickle.dump(glovePretrainedModel, fp)

glovePretrainedModel = download_gloveAlgorithm()

In [ ]:
def getAverage_attachedLabel(labels, dimension, attaching_index):
    labelAttached_container = []
    for token in labels:
        sumOf_attachedLAbels = np.zeros(shape=(dimension, ))
        for tkn in token:
            getVecByToken = attaching_index.get_vecs_by_tokens([tkn]).asnumpy().reshape(-1)

            sumOf_attachedLAbels += getVecByToken
            
        avg_embedding = sumOf_attachedLAbels/len(token)
        labelAttached_container.append(avg_embedding.tolist())

    labelAttached_container = np.array(labelAttached_container)
    
    return labelAttached_container

In [ ]:
averageAttachedLabel = getAverage_attachedLabel(labels, 
                                                50, 
                                                glovePretrainedModel)
averageAttachedLabel.shape

In [ ]:
def distance_betweenPoints(a_point, b_point, axis=None):#eucledean distance
    return np.sqrt( np.power(a_point - b_point, 2).sum(axis=axis) )

def multiDimensionalAverageArray_ofTheCluster(element, cluster=3):#cetroid
    mean, observation = np.shape(element)
    centroids = np.mat( np.zeros((cluster, observation)) )

    minimizeDimension = np.min(element, axis=0)
    maximizeDimension = np.max(element, axis=0) - minimizeDimension

    multiDimensionalAverage_ofTheCluster = np.tile(minimizeDimension, (cluster, 1)) + np.multiply(np.tile(maximizeDimension, (cluster, 1)), np.random.rand(cluster, observation))

    return multiDimensionalAverage_ofTheCluster

In [ ]:
def k_meansClustering(element, cluster, calculateEuclideanDistance=distance_betweenPoints, createClusterMean=multiDimensionalAverageArray_ofTheCluster):
    mean, observations = np.shape(element)
    newContainer_forCluster = np.mat( -np.ones((mean, 2)) )
    ClusterMean_centroid = createClusterMean(element, cluster)
    modified = True

    while modified:
        modified = False

        for each_element in range(mean): 
            minDistance = -1 
            minIndex = -2

            for each_subElement in range(cluster):
                EuclideanDistanceCalculation = calculateEuclideanDistance( element[each_element, :], ClusterMean_centroid[each_subElement, :] )
                if EuclideanDistanceCalculation < minDistance or minDistance == -1:
                    minDistance = EuclideanDistanceCalculation; minIndex = each_subElement

            if newContainer_forCluster[each_element, 0] != minIndex: modified = True
            newContainer_forCluster[each_element, :] = (minIndex, minDistance**2)

        for centreCluster in range(cluster):  
            pointsPatternOfCluster = element[ np.nonzero(newContainer_forCluster[:, 0].A == centreCluster)[0] ]  
            ClusterMean_centroid[centreCluster, :] = np.mean(pointsPatternOfCluster, axis=0)
    return ClusterMean_centroid, newContainer_forCluster

In [ ]:

def divisiveHierarchyClustering_kMeans(element, cluster, calculateEuclideanDistance=distance_betweenPoints):
    mean, observations = np.shape(element)
    multiDimensionalAverage_centroid = np.mean(element, axis=0).tolist()[0]
    multiDimensionalAverage_cluster = [multiDimensionalAverage_centroid] 

    newContainer_forCluster = np.mat( np.zeros((mean, 2)) )
    for each_element in range(mean):
        newContainer_forCluster[each_element, 1] = calculateEuclideanDistance(multiDimensionalAverage_centroid, element[each_element, :]) ** 2

    while(len(multiDimensionalAverage_cluster) < cluster):
        lowest_SumOFSquaredError = -1
        for each_element, _ in enumerate(multiDimensionalAverage_cluster):
            subCluster = element[np.nonzero(newContainer_forCluster[:, 0].A == each_element)[0], :]  

            subCentroids, subContainer_forCluster = k_meansClustering(subCluster, 2, calculateEuclideanDistance)
            if np.any( np.isnan(subCentroids) ) == True:
                subCentroids, subContainer_forCluster = k_meansClustering(subCluster, 2, calculateEuclideanDistance)
                if np.isnan( np.sum(subCentroids) ) == True:
                    continue

            subSquaredError = np.sum( subContainer_forCluster[:, 1] )
            nonZero_subSquaredError = np.sum( newContainer_forCluster[np.nonzero(newContainer_forCluster[:, 0].A != each_element)[0], 1] )
            if (subSquaredError + nonZero_subSquaredError) < lowest_SumOFSquaredError or lowest_SumOFSquaredError == -1:
                lowest_SumOFSquaredError = subSquaredError + nonZero_subSquaredError
                optimalSplit_centre = each_element
                optimal_subcentroids = subCentroids
                optimalContainer_forCluster = subContainer_forCluster

        if lowest_SumOFSquaredError == -1: break 
        print("{} multiDimensionalAverage_cluster of SumOFSquaredError: {}".format(len(multiDimensionalAverage_cluster)+1, lowest_SumOFSquaredError))
        indx1 = np.nonzero(optimalContainer_forCluster[:, 0].A == 1)[0]
        indx0 = np.nonzero(optimalContainer_forCluster[:, 0].A == 0)[0]
        optimalContainer_forCluster[indx1, 0] = len(multiDimensionalAverage_cluster)
        optimalContainer_forCluster[indx0, 0] = optimalSplit_centre
        newContainer_forCluster[np.nonzero(newContainer_forCluster[:, 0].A == optimalSplit_centre)[0], :] = optimalContainer_forCluster

        multiDimensionalAverage_cluster[optimalSplit_centre] = optimal_subcentroids[0].tolist()[0] 
        multiDimensionalAverage_cluster.append(optimal_subcentroids[1].tolist()[0])
    return np.mat(multiDimensionalAverage_cluster), newContainer_forCluster
#the credit for the idea of this model is Jameson's

In [ ]:
meanOfAllPoints, newContainerForCluster = divisiveHierarchyClustering_kMeans(averageAttachedLabel, 100)
print("shape of centroids : ---->", meanOfAllPoints.shape)
print('\n')
print("centroids : ---->", meanOfAllPoints)
print('\n')
print('\n')
print("shape of newContainerForCluster : ---->", newContainerForCluster.shape)
print('\n')
print("newContainerForCluster : ---->", newContainerForCluster)
# DISPLAY

In [ ]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(newContainerForCluster, x=0, y=1, z=0,
              color=0, size=1, size_max=18,
              symbol=1, opacity=0.7)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
#Create submission for kaggle
newContainerForCluster=pd.DataFrame(newContainerForCluster) 
meanOfAllPoints=pd.DataFrame(meanOfAllPoints) 
# #Save submission to CSV
newContainerForCluster.to_csv('submission.csv', index=False)  
meanOfAllPoints.to_csv('submission.csv', index=False)  